# Analisis de posibilidad de join entre datos sernapesca

### Import libreries

In [1]:
import logging
import subprocess
import os   
import pandas as pd
import seaborn as sns

In [2]:
%%time
reporte_plantas_raw = pd.read_excel("../data/datos_sernapesca/Reporte de Plantas.xlsx")
ingresos_raw = pd.read_excel("../data/datos_sernapesca/Ingresos 2021-2022.xlsx")
RNA_raw = pd.read_excel("../data/datos_sernapesca/RNA.xlsx")

CPU times: total: 45.3 s
Wall time: 45.7 s


### Filtrando por región y tipo de producto

In [3]:
# Se filtra por region, no es necesario hacer filtro por producto ya que solo se encuentra 
# info disponible para moluscos
reporte_plantas_filtered = reporte_plantas_raw[(reporte_plantas_raw['Región Bodega'] == 'X REGION') |
                                        (reporte_plantas_raw['Región Agente'] == 'X REGION')]

# Se filtra sólo por tipo de especie = molusco, ya que este dataset ya era sólo de la décima región
ingresos_filtered = ingresos_raw[ingresos_raw['Tipo Especie'] == 'MOLUSCOS']

# El registro nacional de acuicultura de sernapesca (RNA) se encuentra para todo chile, por lo que se filtra
# para la décima región. También se filtra por aquellos centros que se encuentran inscritos.
RNA_filtered = RNA_raw[(RNA_raw['Región'] == 10) & (RNA_raw['Categoria'] == 'INSCRITO')]


In [4]:
shape_reportes =  reporte_plantas_filtered.shape
shape_ingresos =  ingresos_filtered.shape
shape_RNA =  RNA_filtered.shape
print(f'''El dataset de reportes filtrado tiene una forma {shape_reportes}, 
        mientras que el de ingresos es de la forma {shape_ingresos} y 
        el de RNA es de {shape_RNA}''')

El dataset de reportes filtrado tiene una forma (52011, 35), 
        mientras que el de ingresos es de la forma (7998, 22) y 
        el de RNA es de (2338, 18)


Se tienen 3 datasets y queremos saber si es que es posible hacer join entre los 3. Para esto y en primera instancia es necesario definir los campos que podrían ser utilizados para el join. 

* Por parte de la tabla de ingresos, código de agente parece ser una buena opción y también código del último proovedor.

* Para el caso de la tabla de reportes, existe el codigo de centro y el codigo de origen, los cuales podrían servir.

* Finalmente la tabbla de RNA contiene el código del centro de cultivo acuícola.

### Join entre reporte de plantas y RNA

In [5]:
#ingresos_filtered

RNA_list_CCAA = RNA_filtered['Codigo Centro'].unique().tolist()
reporte_plantas_list_origen = reporte_plantas_filtered['Código Origen'].unique().tolist()

In [6]:
results = {}
for i in RNA_list_CCAA:
    results[i] = reporte_plantas_list_origen.count(i) 

In [7]:
df_RNA_reporte_plantas = pd.DataFrame.from_dict(results,orient='index', columns=['count'])
#type(df_RNA_reporte_plantas['count'])
ccaa_match_with_reporte = df_RNA_reporte_plantas[df_RNA_reporte_plantas['count'] > 0]
ccaa_match_with_reporte


,count
100022,1
100242,1
100339,1
101000,1
101011,1
...,...
104342,1
104347,1
104349,1
104366,1


In [15]:
join_RNA_reportes = pd.merge(RNA_filtered, reporte_plantas_filtered, how='left', left_on='Codigo Centro', right_on='Código Origen')
join_RNA_reportes

,Región,Sector,Nm_Provincia,Comuna Centro,Nombre Area,Codigo Centro,Nombre Centro,Categoria,N° Folio RNA,Rut,...,utilizado,Toneladas Rest,Unidades Rest,Tipo Declaración,idBodega,Nombre Bodega,Tipo Bodega,Dirección Bodega,Ciudad Bodega,Región Bodega
0,10,RÌO PUDETO,Chiloé,Ancud,NaN,100009,*,INSCRITO,624.0,84913200-8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,"ESTERO CHOPE, ISLA PULUQUI, SECTOR 1",Llanquihue,Calbuco,NaN,100019,*,INSCRITO,6981.0,69220600-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,CALETA RILAN,Chiloé,Castro,NaN,100022,ENSENADA RILAN,INSCRITO,3162.0,84764200-9,...,-1,0.0,-,Planta Abastecimiento,2719.0,PATIO,PLANTA,AUCAR SN,Ancud,X REGION
3,10,CALETA RILAN,Chiloé,Castro,NaN,100022,ENSENADA RILAN,INSCRITO,3162.0,84764200-9,...,-1,0.0,-,Planta Abastecimiento,2719.0,PATIO,PLANTA,AUCAR SN,Ancud,X REGION
4,10,CALETA RILAN,Chiloé,Castro,NaN,100022,ENSENADA RILAN,INSCRITO,3162.0,84764200-9,...,-1,0.0,-,Planta Abastecimiento,2719.0,PATIO,PLANTA,AUCAR SN,Ancud,X REGION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53720,10,AAMERB BALTAZAR,Palena,Hualaihué,NaN,810248,AAMERB - BALTAZAR,INSCRITO,0.0,65305390-8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53721,10,"PASO QUILOLCURA, NORTE PUNTA LILECURA, ISLA QUENU",Llanquihue,Calbuco,NaN,810262,AAMERB - ISLA QUENU SECTOR B,INSCRITO,0.0,65091450-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53722,10,ESTERO PICHICOLO,Palena,Hualaihué,NaN,810276,AAMERB - ESTE PUNTILLA PICHICOLU,INSCRITO,0.0,65260290-8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53723,10,"CANAL DESERTORES, NORTE CALETA CHANA",Palena,Chaitén,NaN,810286,AAMERB - CHANA SECTOR B,INSCRITO,0.0,74053200-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
